# Answering Planet Climate Questions

In [0]:
!pip install uv

In [0]:
!uv pip install openpyxl
!uv pip install jsonpickle 
!uv pip install --upgrade langchain 
!uv pip install langchain-openai
!uv pip install pydantic==1.10.10
!uv pip install pdfservices-sdk==2.3.1
!uv pip install qdrant-client
!uv pip install --upgrade langchain-core 
!uv pip install --upgrade openai 
!uv pip install langchainhub

In [0]:
dbutils.library.restartPython()

In [0]:
import datetime
import json
import os
import os.path
import re
import sys
import zipfile
from datetime import datetime
from typing import List, Union

import openai
import pandas as pd
import tiktoken
from langchain import LLMChain, OpenAI, PromptTemplate, SerpAPIWrapper
# Langchain imports
from langchain.agents import (AgentExecutor, AgentOutputParser,
                              LLMSingleActionAgent, Tool)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.chains.question_answering import load_qa_chain
# LLM wrapper
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.docstore.document import Document
# Embeddings and vectorstore
from langchain.embeddings.openai import OpenAIEmbeddings
# Conversational memory
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.schema import (AgentAction, AgentFinish, HumanMessage,
                              SystemMessage)
from langchain.text_splitter import TokenTextSplitter
from langchain.tools.render import format_tool_to_openai_function
from langchain.vectorstores import Pinecone
from langchain_core.prompts.prompt import PromptTemplate
from tqdm.auto import tqdm

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
import openai
import os


from functions.adobe import *
from functions.text_processing import * 
from functions.text_prompt import * 
from functions.planet_prompt import *
from functions.tools import *

openai_api_key=os.getenv("OPENAI_KEY_GPT4")
openai_api_base=os.getenv("OPENAI_BASE_GPT4")
openai.api_key = openai_api_key
openai.api_base = openai_api_base
openai.api_type = "azure"
openai.api_version = "2024-02-15-preview"

In [0]:
from langchain.vectorstores import Qdrant
import qdrant_client
from qdrant_client import grpc 
from qdrant_client.conversions.conversion import RestToGrpc
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from openai import AzureOpenAI


In [0]:
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    openai_api_version="2024-02-15-preview",
    openai_api_key=openai_api_key,
    azure_endpoint=openai_api_base,
    temperature=0,
)

In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter
token_splitter = TokenTextSplitter(chunk_size=3000, chunk_overlap=300)

In [0]:
embeddings = AzureOpenAIEmbeddings(
        azure_deployment = "text-embedding-3-large",
        openai_api_version = "2023-05-15",
        model = "text-embedding-3-large",
        api_key = openai_api_key,
        azure_endpoint= openai_api_base
    )

In [0]:
question_groups = {
    "renewable energy target": [
        "The client has set targets for its renewable energy usage",
        "The client has a goal of using 100% renewable energy, regardless of organization growth",
        "The client has met or is on track to meet its renewable energy target"
    ],
    "climate commitments": [
        "The client has (publicly) committed to the Paris Climate Agreement/Net Zero/Decarbonisation to become carbon net zero by 2050 or earlier",
        "The client has developed a climate/decarbonisation strategy including a action plan for reducing carbon emissions from its operations",
        "The climate action plan is externally verified by an independent third-party (e.g. SBTi certified professional, accountant",
        "In the externally verified climate action plan, at least 70% of its portfolio/business activities is in scope",
        "The emission reduction target is considered net-zero and is science-based (i.e. follows the Net Zero Standard of the SBTI or has been approved by the SBTI)"
    ],
    "climate risks": [
    "Climate change physical and transition risks: The client has conducted a climate risk assessment (preferably in line with TCFD recommendations) of its own operations to identify potential transition and physical risks, as well as climate solutions and opportunities",
    "Climate change physical and transition risks: The client has conducted a climate risk assessment (preferably in line with TCFD recommendations) of its supply chain to identify potential transition and physical risks, as well as climate solutions and opportunities"
    ],
    "GHG emissions": [
        "The client monitors and discloses its Scope 1 (operational) GHG emissions",
        "The client monitors and discloses its GHG emissions associated with Scope 2 (purchased electricity)",
        "The client monitors and discloses its Scope 3 emissions (indirect, value chain) either in total or in one or more material sources/categories", ]
}

### Use the question relevant text to answer

In [0]:
from tqdm import tqdm
import pandas as pd


def extract_company_and_year(full_context):
    # Take the first few pages of the full context
    initial_pages = "\n".join([doc.page_content for doc in full_context[:3]])
    
    prompt = f"""
    Given the following text from the first few pages of a sustainability report, identify and return the name of the company that produced this report and the reporting year in a json format. The output must be in JSON format, with the following structure and keys strictly adhered to: 
                {{
                "Company": "The name of the company that produces the report",
                "year": "The reporting year"
                }}  
    Your JSON structure must follow the following principles and this is non-negotiable:
        * Only return valid JSON
        * Never include backtick symbols such as: `
        * The response will be parsed with json.loads(), therefore it must be valid JSON.

    Text:
    {initial_pages}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts company names from sustainability reports."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content.strip()
    

example_template = """
---------question--------- 
{question}
---------context--------- 
{context}
---------answer---------
 {answer}
"""


# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["question", "context", "answer"], template=example_template
)

def create_prompt(topic, question, context):
    if topic in topic_question_data and question in topic_question_data[topic]:
        data = topic_question_data[topic][question]
        few_shot_prompt = FewShotPromptTemplate(
            examples=data["example"],
            example_prompt=example_prompt,
            prefix=f"""
            You are a sustainability reporting specialist with expertise in evaluating corporate environmental practices. Use the following pieces of context to determine whether the statement is correct. If the document does not provide enough context to determine and you are really not sure about the answer, just say that you are unsure.

            Context: {{context}}
            Topic: {topic}
            Statement: {question}
            Specific Instructions: {data['instruction']}

            Here are an example of how to analyze:
            """,
            suffix="""
             Make sure you also read and carefully follow these general instructions:
            - Focus solely on the PARENT COMPANIES, ignoring information from subsidiaries, regions, or countries.
            - The answer may not be straightforward, requiring combining contexts, the use of reasoning skills and professional judgment to interpret the information
            - The output must be in JSON format, with the following structure and keys strictly adhered to: 
                {{
                "OBSERVATION": "Detailed observations based on analyzing the context and a motivation for your responses, keep it concise but also concrete and detailed enough. Don't be vague. Do not just repeat the statement. Use five sentences maximum.",
                "FINAL_ANSWER": 'Yes' or 'No',
                "SOURCES": "EXACT, UNMODIFIED EXCERPTS FROM THE GIVEN CONTEXT that support your findings. These extracts must match the source document character-for-character, including spaces, line breaks, and punctuation. DO NOT ALTER THE TEXT IN ANY WAY.  DO NOT ADD ANYTHING EXTRA. DO NOT INVENT ANYTHING. DO NOT WRITE YOUR OWN OBSERVATION HERE. EVERY STATEMENT MUST BE A VERBATIM QUOTE FROM THE PROVIDED CONTEXT. RETURN ALL RELEVANT REFERENCES you use for producing the answer. If absolutely nothing related can be found, respond with 'No relevant context found.'"

                }}
            - Your JSON structure must follow the following principles and this is non-negotiable:
                * Only return valid JSON
                * Never include backtick symbols such as: `
                * The response will be parsed with json.loads(), therefore it must be valid JSON.
            """,
            input_variables=["context"]
        )
        return few_shot_prompt.format(context=context)


def analyze_question(question, question_relevant_excerpts, topic, question_index):
    context = {
        'context': question_relevant_excerpts,
        # 'history': previous_answers,
    }
    
    prompt = create_prompt(topic, question, context)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content


def create_retriever(documents: list[Document], k, lambda_mult):
    # Create an Embeddings Instance of Azure OpenAI
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment = "text-embedding-3-large",
        openai_api_version = "2023-05-15",
        model = "text-embedding-3-large",
        api_key = openai_api_key,
        azure_endpoint= openai_api_base
    )

    # Create the vector store using Qdrant
    db = Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name="my_documents",
        location=":memory:",
    )
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": k, "lambda_mult": lambda_mult}
    )
    return retriever

def retrieve_question_excerpts(question, full_context, k, lambda_mult):
    """
    Retrieve specific excerpts relevant to the question from the topic context,
    considering previous answers.
    """
    # Create the retriever using the full context
    retriever = create_retriever(full_context, k, lambda_mult)
    
    # Use the retriever to get the most relevant documents
    relevant_docs = retriever.get_relevant_documents(question)
    
    # Concatenate the content of the retrieved documents
    question_relevant_excerpts = "\n\n".join([doc.page_content for doc in relevant_docs])

    return question_relevant_excerpts
    
def process_question_batch(question_groups, full_context,  k, lambda_mult):
    results = []
    company = extract_company_and_year(full_context)
    for topic, topic_questions in question_groups.items():
        # previous_answers = []
        for index, question in enumerate(tqdm(topic_questions)):
            questions_relevant_excerpts = retrieve_question_excerpts(question, full_context, k, lambda_mult)
            analysis = analyze_question(question, questions_relevant_excerpts, topic, index)
            
            results.append({
                "company": company,
                "topic": topic,
                "question": question,
                "analysis": analysis,
                "relevant_excerpts": questions_relevant_excerpts
            })
            
            # Add this question and its answer to previous_answers for the next iteration
            # previous_answers.append({"question": question, "observation": analysis})
    
    return results

In [0]:
import os
from tqdm import tqdm

df_results = pd.DataFrame()

output_base_zip_path = "/tmp/sdk_result/"
output_base_extract_folder = (
    "/dbfs/FileStore/projects/WSsustainability/data/validation/ISA/adobe_raw"
)
all_files = os.listdir(
    "/dbfs/FileStore/projects/WSsustainability/data/validation/ISA/adobe_raw"
)
to_run_file = [os.path.join(output_base_extract_folder, file) for file in all_files]

# Processing each PDF
for pdf_path in tqdm(to_run_file):
    pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
    output_zip_path = os.path.join(output_base_zip_path, f"{pdf_name}.zip")
    output_zipextract_folder = os.path.join(output_base_extract_folder, pdf_name)
    
    try:
        df = extract_text_from_file_adobe(output_zip_path, output_zipextract_folder)
        texts = token_splitter.split_documents([
            Document(page_content=row[1]["text"], metadata={"source": pdf_name, "page": row[1]["page_number"]})
            for row in df.iterrows()
        ])
        results = process_question_batch(question_groups, texts, 7, 0.7)
        df_results_company = pd.DataFrame(results)
        df_results_company['pdf_name'] = pdf_name
        df_results = pd.concat([df_results, df_results_company], ignore_index=True)
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

df_results


In [0]:
# df_results.to_csv("/dbfs/FileStore/projects/WSsustainability/data/validation/Planet/output/Planet_rag_2507.csv",
#     index=False)


In [0]:
import re
import json
import pandas as pd

def clean_and_parse_json(json_str):
    if not isinstance(json_str, str):
        print(f"Input is not a string: {type(json_str)}")
        return None

    # Remove leading/trailing whitespace
    json_str = json_str.strip()

    # Check if the JSON string is wrapped with ```json or ```
    if json_str.startswith('```json'):
        json_str = json_str.replace('```json\n', '').replace('\n```', '')
    elif json_str.startswith('```'):
        json_str = json_str.replace('```\n', '').replace('\n```', '')

    # Escape single quotes
    json_str = json_str.replace("\\'", "'")
    
    # Remove newline characters and extra spaces
    json_str = re.sub(r'\s+', ' ', json_str)

    # Ensure proper JSON format: remove trailing commas and fix curly braces
    json_str = re.sub(r',\s*}', '}', json_str)
    json_str = re.sub(r',\s*]', ']', json_str)

    # Try to parse the JSON string
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        # If JSON parsing fails, return the original string wrapped in a dictionary
        return {"original_text": json_str}

def parse_json_columns(df, json_columns):
    for col in json_columns:
        print(f"Processing column: {col}")
        # Clean and parse the JSON data
        parsed_data = df[col].apply(lambda x: clean_and_parse_json(x))
        
        # Normalize the parsed JSON data
        try:
            normalized_data = pd.json_normalize(parsed_data)
            # Concatenate the normalized data back to the original DataFrame
            df = pd.concat([df.drop(columns=[col]), normalized_data], axis=1)
        except Exception as e:
            print(f"Error normalizing data for column {col}: {e}")
    
    return df

# Specify the columns containing JSON data
json_columns = ['company', 'analysis']

# Parse the JSON columns
df_parsed = parse_json_columns(df_results, json_columns)
df_parsed

In [0]:
df_parsed.to_csv("/dbfs/FileStore/projects/WSsustainability/data/validation/Planet/output/Planet_rag_3107.csv",
    index=False)


## Get url

In [0]:
import pandas as pd
df_url = pd.read_excel('/dbfs/FileStore/projects/WSsustainability/data/validation/ISA/search/isa_report_url_validation.xlsx', sheet_name='in')
selected_rows = ['Yes','yes/?', '_x000D_Yes', '_x001D_Yes' ]
df_url = df_url[df_url['Yes/No/?'].isin(selected_rows)]

df_url['filename'] = df_url['encoded_url'].str.split('/').str[-1].str.replace('.pdf', '').str.replace(' ', '_').str.replace('-', '_')
df_url = df_url[['encoded_url', 'filename']]
df_url = df_url.drop_duplicates(subset = 'filename', keep = 'first')
df_planet = pd.read_csv(
    "/dbfs/FileStore/projects/WSsustainability/data/validation/Planet/output/Planet_rag_3107.csv"
)
df_planet['filename']=df_planet['pdf_name'].str.replace(' ', '_').str.replace('-', '_')
df_output = pd.merge(df_planet, df_url, left_on='filename', right_on='filename', how='left')
df_output = df_output.drop_duplicates()
df_output

In [0]:
df_output.to_csv("/dbfs/FileStore/projects/WSsustainability/data/validation/Planet/output/Planet_rag_3107.csv",
    index=False)

